### 此程序用于《毕业论文》3.1榆中站气象要素的年际、月际变化特征分析
---
### 前期数据处理流程
1. 原始数据为http://weather.uwyo.edu/upperair/seasia.html 获得的FSL文件
2. 经过read_file_day.pro处理后得到每次探测数据的分离文件，得到FSL（文件夹）数据
3. 经过Output_Yuzhong_2010_2018_As_MWMOD_Input_Format.pro处理后，转换成相对湿度并按照高度给定高度插值，得到FSL1（文件夹）数据
### 本程序完成的后续工作
4. 从FSL1数据中提取某些气象要素的年际、月际变化信息
5. 绘制垂直高度剖面图
### 关键步骤
- 异常值处理：删除该观测时间点数据
- 注意修改取样日期、时间、横轴坐标宽度

In [181]:
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import calendar

In [324]:
# 数据搜索
target_str = '2009'
Ti = '00'
dirpath = r'I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1'
filepaths = []
for root,dirs,files in os.walk(dirpath):
    for file in files:
        if((len(target_str)==4)and(target_str == file[30:34])and(file[38:40]==Ti)):
            filepaths.append(os.path.join(root,file))
        elif((len(target_str)==6)and(target_str == file[30:36])and(file[38:40]==Ti)):
            filepaths.append(os.path.join(root,file))
# print(len(filepaths))
# for filepath in filepaths:
#     print(filepath)

In [325]:
## 数据预处理
drawing_data = pd.DataFrame()
head_col = ['Press','Altitude','Temperature','Relative_humidity','NaN1','NaN2','NaN3']
# 数据框架
data = pd.read_table(filepaths[0],sep='\s+',skiprows=6,names=head_col)
print(data.columns)
drawing_data['Altitude'] = data['Altitude']
drawing_data[filepaths[0][88:98]] = data['Temperature']
for time in range(1,len(filepaths)-1):
    data = pd.read_table(filepaths[time],sep='\s+',skiprows=6,names=head_col)
    if(data.mean()['Temperature'] >= 0):
        drawing_data[filepaths[time][88:98]] = data['Temperature']
# drawing_data.replace([-99999,-9999.99023],[0,0],inplace=True)
# drawing_data = drawing_data.round(2)
# print(calendar.monthrange(int(target_str[0:4]),int(target_str[4:6]))[1])
drawing_data.shape[1]

Index(['Press', 'Altitude', 'Temperature', 'Relative_humidity', 'NaN1', 'NaN2',
       'NaN3'],
      dtype='object')


365

In [326]:
drawing_data.to_csv(r'H:\桌面\test.csv')

In [327]:
# 剖面图绘制
fig = plt.figure(figsize=(5,4),dpi=800,frameon=False) #新建画布
ax = fig.add_axes([0,0,4,2])
# 坐标轴标签
ax.set_xlabel('Date',fontsize=20)
ax.set_ylabel('Altitude km',fontsize=20)
drawing_data = drawing_data.iloc[:,1:]
nor = matplotlib.colors.Normalize(vmin=190, vmax=310)
ca = ax.contourf(drawing_data,10,cmap='jet',norm=nor)
# 图例
plt.colorbar(ca)
# X轴坐标
x_ticks = np.arange(0,drawing_data.shape[1],50)
x_ticks = np.append(x_ticks,drawing_data.shape[1]-1)
x_labels = []
for i in range(len(x_ticks)):
    x_labels.append(drawing_data.columns[x_ticks[i]][4:8])
print(x_ticks,x_labels)
plt.xticks(x_ticks,x_labels,rotation=45,fontsize=20)
# Y轴坐标
y_ticks = [0,2,4,6,8,10,14,18,22,26,30,34,38,42,46]
y_labels = ['0','0.2','0.4','0.6','0.8','1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10.0']
plt.yticks(y_ticks,y_labels,fontsize=20)
# 图表标题
# plt.title(target_str+'_Ti'+Ti,fontsize=30)
# 储存图片
picture_path = r'I:\Data\Personal Data\graduation project\yuzhongzhan\picture' +'\\'+ target_str+'_Ti'+Ti+ '.svg'
plt.savefig(picture_path,bbox_inches = 'tight')
# 展示图片
plt.show()

[  0  50 100 150 200 250 300 350 363] ['0101', '0220', '0411', '0531', '0720', '0908', '1028', '1217', '1230']
